# <span style='color:Blue'> Vector Space Model with a Champion List Implementation </span>

- #### The aim is to create an Information Retrieval model able to retrieve documents, given a 'free-form' query in input through an union of the champion lists of the single terms contained in the query.
- #### After the first query, the user can express his preferences, then the Rocchio algorithm moves the query in the direction desired by the user.

## Just some useful imports

In [ ]:
from collections import defaultdict
from itertools import islice
from math import log, sqrt
from functools import reduce
import csv
import re
import pickle

## Let's rertrieve the corpus and store it in a dictionary!

The good old csv module do the job as seen in class

In [ ]:
class MovieDescription:
    
    def __init__(self,docID, title, description):
        self.title = title
        self.description = description
        self.docID = docID
        
    def __repr__(self):
        return self.title

def readMovieDescriptions():
    filename = 'MovieSummaries/plot_summaries.txt'
    movie_names_file = 'MovieSummaries/movie.metadata.tsv'
    with open(movie_names_file, 'r') as csv_file:
        movie_names = csv.reader(csv_file, delimiter='\t')
        names_table = {}
        for name in movie_names:
            names_table[name[0]] = name[2]
    with open(filename, 'r') as csv_file:
        descriptions = csv.reader(csv_file, delimiter='\t')
        corpus = []
        for docID, desc in enumerate(descriptions):
            try:
                movie = MovieDescription(docID, names_table[desc[0]], desc[1])
                corpus.append(movie)
            except KeyError:
                pass
        return corpus

## Exploring the corpus...

In [ ]:
corpus = readMovieDescriptions()
corpus

In [ ]:
corpus[0].description

## Mhh... we need to normalize the description of the movies

The good old re module do the job as seen in class

In [ ]:
def normalize(text):
    no_punctuation = re.sub(r'[^a-zA-Z\s]+','',text)
    downcase = no_punctuation.lower()
    return downcase

def tokenize(text):
    text = normalize(text)
    return list(text.split())

In [ ]:
for movie in corpus:
    movie.description, movie.title = tokenize(movie.description), tokenize(movie.title)

## Let's create an Inverted Index and compute Term Frequency for each docID associated with the current Term
- #### We'll need it to compute tf-idf and create the whole space vector in an efficient way (this is the hope...).
- #### First of all we need to deal with the title and description of the movies, then create an inverted index.
#### (For the sake of simplicity, i merged together title and description).

In [ ]:
def makeInvertedIndex(corpus):
    """
    Each posting is not only a document id, but the term frequency
    where the term is contained in the article.
    
    We are creating an entity of this type {term: {docID: termFrequency}}
    """
    index = defaultdict(dict)
    for docID, _ in enumerate(corpus):
        for term in corpus[docID].title + corpus[docID].description:
            try:
                index[term][docID] += 1
            except KeyError:
                index[term][docID] = 1
    return index, docID+1


In [ ]:
inv_index, length_corpus = makeInvertedIndex(corpus)

In [ ]:
inv_index['hello']

## Creating an Inverted Index containing for each term, the 15 most relevant docIDs (sorted by Term Frequency) -> ChampionLists

In [ ]:
def makeInvertedIndexChampionList(inv_index):

    inv_index_champList = defaultdict(list)
    max_length_champList = 15

    """
    We are creating an entity of this type {term: [docID1, docID2, docID3...]}
    """
    
    for term in inv_index:
        sorted_dict = {docID: tf for docID, tf in sorted(inv_index[term].items(), key=lambda item: item[1], reverse=True)}
        if len(sorted_dict) > max_length_champList:
            for i in range(0,max_length_champList):
                docID = list(sorted_dict.keys())[i]
                inv_index_champList[term].append(docID)
        else:
            for i in range(0,len(sorted_dict)):
                docID = list(sorted_dict.keys())[i]
                inv_index_champList[term].append(docID)
    return inv_index_champList

In [ ]:
champList = makeInvertedIndexChampionList(inv_index)

In [ ]:
champList['hello']

## Creating an Inverted Index with the tfidf for each document of each term 
## -> {term: {docID: tfidf}}

In [ ]:
def makeInvertedIndex_tfidf(inv_index, length_corpus):
    N = length_corpus
    inv_index_tfidf = defaultdict(dict)
    
    for term in inv_index.keys():
        idf = log(N/len(inv_index[term]))
        inv_index_tfidf[term] = {docID: tf * idf for docID, tf in inv_index[term].items()}
    return inv_index_tfidf

In [ ]:
inv_index_tfidf = makeInvertedIndex_tfidf(inv_index, length_corpus)

In [ ]:
inv_index_tfidf['taxi']

## Let's investigate the term 'taxi' in the list regarding the first document in the corpus

In [ ]:
docID = 0 ## first document
term = 'taxi' ## first term object
print("term:", term, "tfidf_docID_0:", inv_index_tfidf[term][docID])

## Our Inverted Index represent the whole Vector Space in an efficient way!!!
- #### Represent a vector of the Vector Space for the first document -> {Term: tf-idf}
- #### Absence of a term in the vector -> tfidf = 0 -> Compact Representation!!!

In [ ]:
def documentToVector(docID, inv_index_tfidf):
    vector = {}

    for term in inv_index_tfidf.keys():
        try:
            vector[term] = inv_index_tfidf[term][docID]
        except KeyError:
            pass
    return vector

In [ ]:
docID = 0
vector = documentToVector(docID, inv_index_tfidf)
vector

## We can do better, sort by tfidf and normalize the vector!

In [ ]:
def sortVector(vector):
    sorted_vector = {k: v for k, v in sorted(vector.items(), key=lambda item: item[1], reverse=True)}
    return sorted_vector

def normalizeVector(vector):
    length = sqrt(sum([x**2 for x in vector.values()]))
    normalized = {k: tfidf/length for k, tfidf in vector.items()}
    return normalized

def sortAndNormalize(vector):
    return sortVector(normalizeVector(vector))

In [ ]:
vector = sortAndNormalize(vector)
vector

## Let's create a a method to parse a query of terms in a normalized vector

In [ ]:
def queryAsVector(query):
    query = tokenize(query)
    query_vector = {}

    for term in query: #iterate through all the query terms
        query_vector[term] = 1
    query_vector = normalizeVector(query_vector)
    return query_vector

In [ ]:
query = "christmas murder love"
query_vector = queryAsVector(query)
query_vector

## Creating the VectorSpace

In [ ]:
def createVectorSpace(inv_index_tfidf, length_corpus):
    vectorSpace = defaultdict(dict)
    for term in inv_index_tfidf.keys():
        for docID in inv_index_tfidf[term].keys():
            vectorSpace[docID][term] = inv_index_tfidf[term][docID]
    return vectorSpace

In [ ]:
vectorSpace = createVectorSpace(inv_index_tfidf, length_corpus)

In [ ]:
vectorSpace[0]

## Now we need to normalize the whole Vector Space

In [ ]:
def normalizeVectorSpace(vectorSpace):
    for docID, vector in vectorSpace.items():
        vectorSpace[docID] = sortAndNormalize(vectorSpace[docID])
    return vectorSpace

In [ ]:
vectorSpace = normalizeVectorSpace(vectorSpace)

In [ ]:
vectorSpace[12]

## Let's create the Inverted Index of the normalized Vector Space 😃

In [ ]:
def makeInvertedIndexNormalized(vectorSpace):
    inv_index_normalized = defaultdict(dict)
    for docID in vectorSpace.keys():
        for term, tfidf_normalized in vectorSpace[docID].items():
            inv_index_normalized[term][docID] = tfidf_normalized
    return inv_index_normalized

In [ ]:
inv_index_normalized = makeInvertedIndexNormalized(vectorSpace)

In [ ]:
inv_index_normalized['taxi']

## Now we have all the ingredients to compute the Cosine similarity between a query represented as a vector and the Vector Space!!
- #### Cosine similarity between normalized vectors -> Inner Product
- #### Here an example of inner product between the query vector and another document

In [ ]:
def innerProduct(vectorA, vectorB):
    setA = set(vectorA.keys())
    setB = set(vectorB.keys())
    product = 0
    intersection = setA.intersection(setB)
    
    for term in intersection:
        product += vectorA[term] * vectorB[term]
    return product

In [ ]:
query = "christmas murder love"
query_vector = queryAsVector(query)

vector = vectorSpace[10]

innerProduct(query_vector, vector)

## Search for the best answer to a given query in the whole VectorSpace... 🤖
- Compute Inner Product between a query and every document of the vectorSpace -> Compute a search on the normalized Inverted Index!!!
- sort results by value of the Inner Product

In [ ]:
query = "christmas murder love"
query_vector = queryAsVector(query)
query_vector

In [ ]:
result_products = defaultdict(dict)

for term, query_tfidf in query_vector.items():
    for docID, tfidf_normalized in inv_index_normalized[term].items():
        result_products[docID][term] = tfidf_normalized * query_tfidf
for docID, vector_products in result_products.items():
    print(docID, vector_products)

In [ ]:
result_innerProduct = {}
for docID, vector_products in result_products.items():
    result_innerProduct[docID] = sum(vector_products.values())
result_innerProduct

In [ ]:
result_innerProduct = sortVector(result_innerProduct)
result_innerProduct

In [ ]:
result_titles = {}
for docID in result_innerProduct.keys():
    result_titles[docID] = ' '.join(corpus[docID].title)
result_titles

## End of the show, wrap everything in a function!
- #### As we can see, we have a search that has a time complexity O(#terms in query * #documents where the current term is present)

In [ ]:
def searchVectorSpaceAsInvertedIndex(query_vector, inv_index_normalized):
    result_products = defaultdict(dict)

    for term, query_tfidf in query_vector.items():
        for docID, tfidf_normalized in inv_index_normalized[term].items():
            result_products[docID][term] = tfidf_normalized * query_tfidf

        result_innerProduct = {}
        for docID, vector_products in result_products.items():
            result_innerProduct[docID] = sum(vector_products.values())
        result_innerProduct = sortVector(result_innerProduct)

        result_titles = {}
        for docID, inner_product in result_innerProduct.items():
            result_titles[docID] = ' '.join(corpus[docID].title)
    return result_titles

In [ ]:
searchVectorSpaceAsInvertedIndex(query_vector, inv_index_normalized)

## -> Let's try to use the vectorSpace for the search instead!!

In [ ]:
query = "christmas murder love"
query_vector = queryAsVector(query)
query_vector

In [ ]:
result_innerProduct = {}
for docID, current_vector in vectorSpace.items():
    inner_product = innerProduct(query_vector, current_vector)
    if inner_product > 0:
        result_innerProduct[docID] = inner_product
result_innerProduct

In [ ]:
result_innerProduct = sortVector(result_innerProduct)
result_innerProduct

In [ ]:
result_titles = {}
for docID in result_innerProduct.keys():
    result_titles[docID] = ' '.join(corpus[docID].title)
result_titles

## End of the show, wrap everything in a function!
- #### As we can see, we have a search that has a time complexity O(#documents in the Vector Space * #terms contained in the current document)

In [ ]:
def docIDListToTitles(result):
    res_titles = {docID: ' '.join(corpus[docID].title) for docID in result}
    return res_titles

def searchVectorSpace(query_vector, vectorSpace):
    result_innerProduct = {}
    for docID, current_vector in vectorSpace.items():
        inner_product = innerProduct(query_vector, current_vector)
        if inner_product > 0:
            result_innerProduct[docID] = inner_product
    result_sorted_by_innerProduct = sortVector(result_innerProduct)
    docID_list = list(result_sorted_by_innerProduct.keys())
    result_titles = docIDListToTitles(docID_list)
    return result_titles

In [ ]:
searchVectorSpace(query_vector, vectorSpace)

## OK! Everything is working, let's work with the Champion Lists!!!
- #### The champList entity it's of the type {term: [docID1, docID14, docID23, ...]}

## Responding to a query using the CampionList... 🤖

In [ ]:
def union(listA, listB):
    setA = set(listA)
    setB = set(listB)
    union = setA.union(setB)
    return list(union)

def searchChampionList(query, champList):
    query = tokenize(query)
    result_list = []
    
    for term in query:
        result_list.append(champList[term])
    union_result_list = reduce(union, result_list)
    return docIDListToTitles(union_result_list)

In [ ]:
query = "christmas murder love"
searchChampionList(query, champList)

## Store the ChampList and the vector Space

In [ ]:
def saveObject(obj, name):
    with open('objects/' + name + '.pkl', 'wb') as outfile:
        pickle.dump(obj, outfile, pickle.HIGHEST_PROTOCOL)

def loadObject(name):
    with open('objects/' + name + '.pkl', 'rb') as infile:
        return pickle.load(infile)

In [ ]:
saveObject(champList, "champList")
saveObject(vectorSpace, "vectorSpace")
saveObject(corpus, "corpus")
saveObject(inv_index_normalized, "inv_index_normalized")